In [30]:
from difflib import SequenceMatcher
import pandas as pd
import numpy as np
import json
import string
import re

from openpyxl import load_workbook

In [31]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


def clean_name(name):
    if 'NY-NEW YORK-' in name:
        name = name.split('NY-NEW YORK-')[1]    
    elif name.count('-') == 2:
        name = name.split('-')[1]
    elif name.count('-') == 1:
        if len(name.split('-')[0]) == 2:
            name = name.split('-')[1]
        elif any(map(str.isdigit, name.split('-')[1])):
            name = name.split('-')[0]
    name = name.lower()
    name = name.translate(
        str.maketrans('', '', string.punctuation.replace("&", '')))
    name = name.replace('inc', '')
    name = name.replace("llc", '')
    name = name.replace("the", '')
    name = name.replace("  ", ' ')
    name = name.replace("  ", ' ')
    name = name.strip()
    name = re.sub("\\[a-z]*", '', name)
    return name

Loading Worksheets

In [32]:
# wb = load_workbook(filename='./sheets/October Adfire Data.xlsx')
# wb_1 = load_workbook(filename='./sheets/Target_Contacts.xlsx')
ad_fire_raw = pd.read_excel(
    './sheets/Vatica_22_2022_11_27_weekly_111105.xlsx')
target_raw = pd.read_excel(
    './sheets/Vatica Target Account List.xlsx')

print(ad_fire_raw.shape[0])
print(target_raw.shape[0])

111105
1575


In [33]:
ad_fire_raw['Full_Name'] = ad_fire_raw["First Name"].str.cat(ad_fire_raw["Last Name"], sep = " ")
print(ad_fire_raw.columns)
print(ad_fire_raw['EventType'].unique())
ad_fire = ad_fire_raw[['NPI', 'Full_Name', 'Hospital_Name', 'EventType']]
print(ad_fire.columns)
print(ad_fire.shape[0])

Index(['NPI', 'First Name', 'Last Name', 'City', 'State', 'Zip', 'Title',
       'Specialty', 'Hospital_Name', 'EventType', 'logentrytime',
       'Partner_Name', 'Ad_Group_Name', 'Campaign_Name', 'Creative_Name',
       'adformat', 'browser', 'frequency', 'supplyvendor', 'foldposition',
       'ipaddress', 'referrer_url', 'redirect_url', 'devicetype', 'os',
       'osfamily', 'recency', 'site', 'referrercategorieslist', 'devicemake',
       'renderingcontext', 'temperaturebucketstartincelsiusname',
       'temperaturebucketendincelsiusname', 'userhourofweek', 'Device_ID',
       'Report_id', 'Full_Name'],
      dtype='object')
['Impression' 'Click']
Index(['NPI', 'Full_Name', 'Hospital_Name', 'EventType'], dtype='object')
111105


In [34]:
ad_fire = ad_fire.groupby(ad_fire.columns.tolist(), as_index=False, dropna=False).size()
ad_fire = ad_fire.rename(columns={"size": "EventCount"})
ad_fire.fillna('NaN', inplace=True)
ad_fire = ad_fire.reset_index()
ad_fire

,index,NPI,Full_Name,Hospital_Name,EventType,EventCount
0,0,1003042516,Nagina Aslam,HENRY FORD MACOMB HOSPITAL,Impression,19
1,1,1003073735,Keri Fair,WEST SUBURBAN MEDICAL CENTER,Impression,73
2,2,1003100397,Susmita Senapati,NaN,Impression,14
3,3,1003160714,Lindsi Dearment,LANCASTER GENERAL HOSPITAL,Impression,1
4,4,1003222159,Rita All,SARATOGA HOSPITAL,Impression,10
...,...,...,...,...,...,...
2439,2439,1992799779,Paul McAvoy,NaN,Impression,3
2440,2440,1992807358,Jeffrey Herman,ST JOHN HOSPITAL AND MEDICAL CENTER,Impression,207
2441,2441,1992816615,James Humphrey,ALLEGHENY VALLEY HOSPITAL,Impression,12
2442,2442,1992920771,Michelle Kelly,CHRISTIANA HOSPITAL,Impression,1


In [35]:
print(ad_fire.columns.tolist())
print(ad_fire['EventCount'].sum())

['index', 'NPI', 'Full_Name', 'Hospital_Name', 'EventType', 'EventCount']
111105


In [36]:
print(target_raw.columns)
target = target_raw[['Provider NPI','Full Name', 'Account Name']]
target = target.rename(columns={"Provider NPI": "NPI","Account Name": "Account_Name", "Full Name": "Full_Name"})
target['Full_Name'].fillna('NaN', inplace=True)
target['Account_Name'].fillna('NaN', inplace=True)
target['NPI'].fillna(0, inplace=True)
target = target.reset_index()
print(target.columns)

Index(['When added', 'Created Date', 'Contact 18 digit Id', 'Contact ID',
       '18 Account ID', 'Full Name', 'Title', 'Email', 'Parent Account',
       'Account Name', 'Mailing Street', 'Mailing City',
       'Mailing State/Province (text only)', 'Mailing Zip/Postal Code',
       'Phone', 'Fax', 'Mobile', 'Account Owner', 'Parent Level',
       'Account Level', 'Status', 'Provider NPI', 'Account Record Type',
       'Vatica Site ID', 'Provider Engagement Rep',
       'June Multisource Campaign', 'Closed Won', 'Account Domain',
       'Prioirty'],
      dtype='object')
Index(['index', 'NPI', 'Full_Name', 'Account_Name'], dtype='object')


In [37]:
data_map = {}
impress = 0
clk = 0
for index, row in ad_fire.iterrows():
  if row['EventType'] == 'Impression': 
    impress += row['EventCount']
  else:
    clk += row['EventCount']
  if row['Hospital_Name'] in data_map:
    if row['Full_Name'] in data_map[row['Hospital_Name']]:
      data_map[row['Hospital_Name']][row['Full_Name']][row['EventType']] = row['EventCount']
    else:
      data_map[row['Hospital_Name']][row['Full_Name']] = {
        'NPI': row['NPI'],
        row['EventType']: row['EventCount']
      }
  else:
    data_map[row['Hospital_Name']] = {
      row['Full_Name']: {
        'NPI': row['NPI'],
        row['EventType']: row['EventCount'],
      }
    }

In [38]:
with open('./test_031.json', 'w') as f:
  json.dump(data_map, f, indent=4)

In [39]:
def _clean_name(name):
  name = str(name)
  name = name.translate(
      str.maketrans('', '', string.punctuation))
  name = name.replace("Dr", '')
  name = name.replace("MD", '')
  name = name.replace("RD", '')
  name = name.replace("BSN", '')
  name = name.replace("DO", '')
  name = name.strip()
  return name

In [40]:
target_companies = {}
for index, row in target.iterrows():
  if row['Account_Name'] in target_companies:
    target_companies[row['Account_Name']]['people'].append({row['NPI'] : _clean_name(row['Full_Name'])})
  else:
    target_companies[row['Account_Name']] = {
      'people': [{int(row['NPI']): _clean_name(row['Full_Name'])}]
    }

In [41]:
with open('./test_032.json', 'w') as f:
  json.dump(target_companies, f, indent=4)

In [42]:
def sim(name, name_2):
  name = clean_name(name)
  name_2 = clean_name(name_2)
  cond1 = name in name_2
  cond2 = similar(name, name_2) > 0.75
  cond3 = name.split()[0] == name_2.split()[0]
  cond4 = similar(name, name_2) > 0.9
  return cond1 or (cond2 and cond3) or cond4

In [43]:
ad_fire_comp = list(data_map.keys())
target_comp = list(target_companies.keys())
cross = {}
for t_comp in target_comp:
  for comp in ad_fire_comp:
    if sim(comp, t_comp):
      if t_comp in cross:
        cross[t_comp].append(comp)
      else:
        cross[t_comp] = [comp]

print(len(list(cross.keys())) / len(list(data_map.keys())) * 100)
print(len(list(cross.keys())) / len(list(target_companies.keys())) * 100)

11.875
9.81067125645439


In [44]:
with open('./test_033.json', 'w') as f:
  json.dump(cross, f, indent=4)

In [45]:
def extract(lst):
  names = []
  NPIs = []
  impressions = 0
  clicks = 0
  for name in lst:
    names.append(name)
    NPIs.append(lst[name]['NPI'])
    if 'Impression' in lst[name]:
      impressions += lst[name]['Impression']
    if 'Click' in lst[name]:
      clicks += lst[name]['Click']
  return names, impressions, clicks, NPIs


def t_extract(lst):
  NPIs = []
  names = []
  for person in lst:
    NPIs += list(person.keys())
    names += list(person.values())
  return NPIs, names

In [54]:
detail = {}
total_imp = 0
total_click = 0
total_names = 0
total_target = 0
total_duplicate = 0
all_names_ad_fire = []
all_names_target = []
all_npis = []
all_t_npis = []
for t_comp in list(cross.keys()):
  for comp in cross[t_comp]:
    names, impressions, clicks, NPIs = extract(data_map[comp])
    t_npi, t_names = t_extract(target_companies[t_comp]['people'])
    all_npis += NPIs
    all_t_npis += t_npi
    total_imp += impressions
    total_click += clicks
    total_names += len(names)
    total_target += len(t_names)
    all_names_ad_fire += names
    all_names_target += t_names
    d_names = set(names).intersection(set(t_names))
    total_duplicate += len(list(d_names))
    detail[t_comp] = {
      # 'Names_Ad_Fire': names,
      # 'Names_Target': t_names,
      'Impressions': impressions,
      'Clicks': clicks,
      # 'Duplicate_names': list(d_names),
    }
print("# Impressions: {}\n# Clicks: {}\n# Ad_fire_names: {}\n# Target_names: {}\n# Crossed_names: {}".format(total_imp, total_click, total_names, total_target, total_duplicate))
print("% of total Impressions: {}\n% of total Clicks: {}".format(total_imp / impress * 100, total_click / clk * 100))

all_d_names = set(all_names_ad_fire).intersection(set(all_names_target))
print(len(all_d_names))
print(all_d_names)

all_d_npi = set(all_npis).intersection(set(all_t_npis))
print(len(all_d_npi))
print(all_d_npi)

with open('./test_034_1.json', 'w') as f:
  json.dump(detail, f, indent=4)

# Impressions: 41461
# Clicks: 63
# Ad_fire_names: 766
# Target_names: 311
# Crossed_names: 1
% of total Impressions: 37.35528105882459
% of total Clicks: 55.26315789473685
2
{'Regina Makdissi', 'NaN'}
0
set()


In [47]:
matched = []
for comp in list(cross.values()):
  matched += comp
matched = list(set(matched))

unmatched = [x  for x in list(data_map.keys()) if x not in matched ]
print(len(unmatched), len(matched), len(list(data_map.keys())))
with open('./test_035_0.json', 'w') as f:
  json.dump({'Matched Companies': matched, 'Unmatched Companies': unmatched}, f, indent=4)
with open('./unmatched_new.json', 'w') as f:
  json.dump(unmatched, f, indent=4)

420 60 480


In [48]:
print(clean_name("UNITY HOSPITAL"))
print(clean_name("Hospital IN - Cameron Memorial Community Hospital Hospital - 1415 FLAXMILL RD"))

unity hospital
cameron memorial community hospital hospital


In [49]:
def clean_creative(name):
  if type(name) == float:
    print(name)
    name = str(name)
  if name.count('_') < 4: 
    print(name)
    return 'NaN'
  if name.split('_')[2] == 'EntryPoint1A':
    return name.split('_')[2]
  elif name.split('_')[2].startswith('GTM') or name.split('_')[2].startswith('Entry'):
    return name.split('_')[3]
  else:
    print(name)
    return 'NaN'


In [50]:
creative_raw = ad_fire_raw[['Creative_Name', 'EventType']]
for index, row in creative_raw.iterrows():
  creative_raw.loc[index, ['Creative_Name']] = [clean_creative(row['Creative_Name'])]
creative_raw

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,Creative_Name,EventType
0,Accurate Patient Data,Impression
1,Accurate Patient Data,Impression
2,EntryPoint1A,Impression
3,Accurate Patient Data,Impression
4,EntryPoint1A,Impression
...,...,...
111100,Accurate Patient Data,Impression
111101,EntryPoint1A,Impression
111102,Relieve Admin Burdens,Impression
111103,Accurate Patient Data,Impression


In [51]:
creative_raw.loc[225106:225108]

,Creative_Name,EventType


In [52]:
creative = creative_raw.copy()
creative = creative.groupby(creative.columns.tolist(), as_index=False, dropna=False).size()
creative = creative.rename(columns={"size": "EventCount"})
creative.fillna('NaN', inplace=True)
creative = creative.reset_index()
creative

,index,Creative_Name,EventType,EventCount
0,0,Accurate Patient Data,Click,30
1,1,Accurate Patient Data,Impression,28718
2,2,EntryPoint1A,Click,43
3,3,EntryPoint1A,Impression,39631
4,4,Relieve Admin Burdens,Click,35
5,5,Relieve Admin Burdens,Impression,35337
6,6,Value Based Care,Click,6
7,7,Value Based Care,Impression,7305


In [53]:
creative.to_csv(path_or_buf='./Creative_breakdown_1.csv', index=False)